## 1. Preprocess the reddit dataset

In the following, we will preprocess the Social Network: Reddit Hyperlink Network from [SNAP Stanford](https://snap.stanford.edu/data/soc-RedditHyperlinks.html). The first step preprocess the dataset to a list of `networkX` graphs.

1. Please download the [soc-redditHyperlinks-body.tsv](https://snap.stanford.edu/data/soc-redditHyperlinks-body.tsv) from the repository and move the file to the same directoy as the `preprocess.ipynb` notebook. 
2. You have to also install the Python packages of the `requirements.txt` file. 

In [1]:
import pandas as pd 
import numpy as np
import networkx as nx
from tqdm import tqdm
import pickle

In [2]:
path = 'soc-redditHyperlinks-body.tsv'
df = pd.read_csv(path, sep='\t')

In [3]:
df.drop(['PROPERTIES', 'POST_ID'], axis = 1, inplace = True)
df['TIMESTAMP']= pd.to_datetime(df['TIMESTAMP']) 

In [4]:
unique_subreddit = pd.unique(df[['SOURCE_SUBREDDIT', 'TARGET_SUBREDDIT']].values.ravel('K'))

In [5]:
unique_subreddit

array(['leagueoflegends', 'theredlion', 'inlandempire', ..., 'dildohero',
       'hatedaddyofive', 'ouija_irl'], dtype=object)

In [6]:
keys_dict = {}
for i in np.arange(len(unique_subreddit)): 
    keys_dict[unique_subreddit[i]] = i

In [7]:
df['SOURCE'] = df[['SOURCE_SUBREDDIT']].values
df['TARGET'] = df[['TARGET_SUBREDDIT']].values
df[['SOURCE']] = np.vectorize(keys_dict.get)(df[['SOURCE']].values)
df[['TARGET']] = np.vectorize(keys_dict.get)(df[['TARGET']].values)

In [8]:
# group the graphs into intervals of one hour
df_group = df.groupby([df['TIMESTAMP'].dt.date, df['TIMESTAMP'].dt.hour])

In [9]:
graphs = []

In [10]:
for t, group in df_group:
    G = nx.Graph(time=t)
    for row_index, row in group.iterrows():
        G.add_node(row['SOURCE'], name=row['SOURCE_SUBREDDIT'])
        G.add_node(row['TARGET'], name=row['TARGET_SUBREDDIT'])
        G.add_edge(row['SOURCE'], row['TARGET'], sentiment=row['LINK_SENTIMENT'], time=row['TIMESTAMP'])
    graphs.append(G)

In [11]:
# number of graphs
len(graphs)

28979

In [12]:
# for performance reasons we are only taking the last 2000 graphs here 
graphs = graphs[-2000:]

In [13]:
len(graphs)

2000

__Compute the graph layout__

In [14]:
# G = nx.compose_all(graphs)
G = nx.Graph()
for graph in tqdm(graphs):
    G.add_nodes_from(graph.nodes(data=True))
    G.add_edges_from(graph.edges(data=True))

coordinates = nx.spring_layout(G, iterations=100)

# -- Alternative graph layouts for example --
# coordinates = nx.kamada_kawai_layout(G)
# coordinates = nx.spectral_layout(G)
# coordinates = nx.circular_layout(G)
# coordinates = nx.random_layout(G)

print('Layout done')

100%|██████████| 2000/2000 [00:00<00:00, 12986.93it/s]


Layout done


In [15]:
# coordinates to list     
coordinates = {k: v.tolist() for k, v in coordinates.items()}
# modify positions 
for graph in tqdm(graphs):
    nx.set_node_attributes(graph, coordinates, 'coord')

100%|██████████| 2000/2000 [00:14<00:00, 136.91it/s]


In [16]:
filename = 'reddit_graphs.pkl'
pickle.dump(graphs, open( filename, 'wb' ))

## 2. Compute the graph embeddings

In the second step, the temporal summaries are computed and the graph embeddings. 

In [17]:
from karateclub import Graph2Vec, GL2Vec, FGSD
import itertools
import datetime 
from collections import Counter
import math 

In [18]:
class Snapshot:
    def __init__(self, graphs, indx1, indx2):
        """Initialize snapshot from a list of graphs.

            Keyword arguments:
            graphs -- list of networkX graphs 
            indx1 -- first index in the overall graph list
            indx2 -- last index in the overall graph list 
        """
        self.graphs = graphs[indx1:indx2]
        self.indx1 = indx1
        self.indx2 = indx2
        self.time1 = datetime.datetime.combine(self.graphs[0].graph['time'][0], datetime.time(self.graphs[0].graph['time'][1].item()))
        self.time2 = datetime.datetime.combine(self.graphs[-1].graph['time'][0], datetime.time(self.graphs[-1].graph['time'][1].item()))
        self.duration = self.time2 - self.time1
        self.union_g = None
        
        # occurences of nodes over time in a dict 
        nodes = []
        for g in self.graphs:
            nodes.append(g.nodes)
        # get number of occurences 
        self.node_occ = Counter(x for xs in nodes for x in set(xs))

    def __repr__(self):
        return 'Snapshot: ' + str(self.time1) + ' - ' + str(self.time2) 

    def __str__(self):
        return 'Snapshot: ' + str(self.time1) + ' - ' + str(self.time2) 
    
    def union_graph(self):
        """Return union graph.
        """
        if not self.union_g:
            G = nx.Graph()
            for graph in self.graphs:
                G.add_nodes_from(graph.nodes(data=True))
                G.add_edges_from(graph.edges(data=True))
            self.union_g = G
        return self.union_g
    
    def disjoint_graph(self, num):
        """Return disjoint graph.

            Keyword arguments:
            num -- number of occurences in the sequence of graphs required to be in the disjoint graph (below the number)
        """
        # filter out all occurence values below 1 and return nodes
        nodes_dict = {x : self.node_occ[x] for x in self.node_occ if self.node_occ[x] <= num}
        # union graph 
        union_g = self.union_g
        
        # return the subgraph matching all the nodes dict 
        return union_g.subgraph([*nodes_dict])
    
    def intersection_graph(self, num):
        """Return intersection graph.

            Keyword arguments:
            num -- number of occurences in the sequence of graphs required to be in the intersection graph 
        """
        nodes_dict = {x : self.node_occ[x] for x in self.node_occ if self.node_occ[x] >= num}
        # union graph 
        union_g = self.union_g
        
        # return the subgraph matching all the nodes dict 
        return union_g.subgraph([*nodes_dict])
    
    def get_summaries(self, num):
        """Return all graph summaries in a list.

            Keyword arguments:
            num -- number of occurences in, please see intersection, and disjoint methods 
        """
        return [self.union_graph(), self.disjoint_graph(num), self.intersection_graph(num)]

In [19]:
class Level:
    def __init__(self, graphs, level):
        """Initialize a level from from a list of graphs.

            Keyword arguments:
            graphs -- list of networkX graphs 
            level -- number for the level used to create window size 
        """
        self.graphs = graphs
        self.level = level 
        self.window_size = int(math.pow(2, (level-1)))
        self.overlap = int(self.window_size/2) 
        
        # initialize the snapshots  
        if self.window_size < 1: 
            raise ValueError('Window size of level below 1')
        if self.overlap > 0:
            self.snapshots = []
            # the following if statement should be only false for the highest level of the hierarchy 
            if len(self.graphs) > self.window_size: 
                for i in range(0, len(self.graphs), self.overlap):
                    self.snapshots.append(Snapshot(self.graphs, i, i+self.window_size))
            # root node of hierarchy 
            else: 
                self.snapshots.append(Snapshot(self.graphs, 0, self.window_size))
        else: 
            self.snapshots = self.graphs 
        
    def __repr__(self):
        return 'Level: ' + str(self.level) + ' - '  + str(self.window_size) + ' - ' + str(self.overlap) 

    def __str__(self):
        return 'Level: ' + str(self.level) + ' - '  + str(self.window_size) + ' - ' + str(self.overlap)
    
    def get_graphs(self):
        """Return all graphs of the level. Returns a dict with key:[union_graph, intersection_graph, disjoint_graph].
        The key is the index in the snapshots array.
        """
        print('Level: ' + str(self.level))
        # num = int(self.overlap/2)
        num = 2 # the subreddit hast to at least twice in the dataset
        result = {}
        for index, snap in tqdm(enumerate(self.snapshots)):
            result[index] = snap.get_summaries(num)
        return result

In [20]:
class Hierarchy:
    def __init__(self, graphs):
        """Initialize the hierarchy from a list of graphs.

            Keyword arguments:
            graphs -- list of networkX graphs 
        """
        self.graphs = graphs
        self.levels = {}
        self.height = 1
        
        window = 2
        while window < len(self.graphs):
            self.height = self.height + 1
            window = int(math.pow(2, (self.height-1)))
            self.levels[self.height] = Level(self.graphs, self.height)
        
    def __repr__(self):
        return str(self.levels)

    def __str__(self):
        return self.levels
    
    def get_graphs(self):
        """Return all graphs of the hierarchy. Returns a dict of all graphs and a tuple of keys - refering to the elements.
        """
        result = {}
        for key, value in self.levels.items():
            result[key] = value.get_graphs()
        return result

In [21]:
h = Hierarchy(graphs)

In [22]:
h

{2: Level: 2 - 2 - 1, 3: Level: 3 - 4 - 2, 4: Level: 4 - 8 - 4, 5: Level: 5 - 16 - 8, 6: Level: 6 - 32 - 16, 7: Level: 7 - 64 - 32, 8: Level: 8 - 128 - 64, 9: Level: 9 - 256 - 128, 10: Level: 10 - 512 - 256, 11: Level: 11 - 1024 - 512, 12: Level: 12 - 2048 - 1024}

In [23]:
all_graphs = h.get_graphs()

0it [00:00, ?it/s]

Level: 2


2000it [00:00, 2157.67it/s]
231it [00:00, 2308.44it/s]

Level: 3


1000it [00:00, 1509.72it/s]
122it [00:00, 1205.66it/s]

...

Level: 10


8it [00:00, 26.31it/s]
2it [00:00, 12.10it/s]

Level: 11


4it [00:00, 14.09it/s]
1it [00:00,  5.96it/s]

Level: 12


In [24]:
## helper method
def flatten_dict(dd, separator='_', prefix=''):
    return { str(prefix) + str(separator) + str(k) if prefix else k : v
             for kk, vv in dd.items()
             for k, v in flatten_dict(vv, separator, kk).items()
             } if isinstance(dd, dict) else { prefix : dd }

In [25]:
epochs = 100
num_summary_graphs = 3

In [26]:
# flatten the dict and get the graphs and keys 
all_graphs = flatten_dict(all_graphs)
graph_to_embed = [item for sublist in list(all_graphs.values()) for item in sublist]
graph_keys = list(all_graphs.keys())
# repeat three times as we use three summaries union, disjoint, intersection 
graph_keys = list(itertools.chain.from_iterable(itertools.repeat(x, num_summary_graphs) for x in graph_keys))

In [27]:
len(graph_to_embed)

11997

In [28]:
model = Graph2Vec(workers=16, epochs=epochs)
model.fit(graph_to_embed)
embeddings = model.get_embedding()

In [29]:
#model = GL2Vec(workers=20, epochs=epochs)
#model.fit(graph_to_embed)
#embeddings = model.get_embedding()

In [30]:
#model = FGSD(hist_bins=128)
#model.fit(graph_to_embed)
#embeddings = model.get_embedding()

In [31]:
print(len(embeddings))
print(len(embeddings[0]))

11997
128


In [32]:
result = {'embeddings' : embeddings,
          'keys' : graph_keys
         }
# keys are build with "level_internal"

In [33]:
filename = 'reddit_embeddings.pkl'
pickle.dump(result, open( filename, 'wb'))

## 3. Flask 

The two exportet files `reddit_graphs.pkl` and the `reddit_embeddings.pkl` are then used as inputs to the Flask application. 